In [1]:
import ipywidgets as widgets
from IPython.display import display
import requests

class Chains:
    def __init__(self, user, host):
        self.user = user
        self.host = host
        chains = requests.get(f"{host}/deceptecons/chains").json()
        chains.sort(key=lambda x: x['chid'])
        chains_options = [f"{chain['chid']}: {chain['name']}" for chain in chains]

        self.create_chain_name_text = widgets.Text(
            value='',
            placeholder='Enter the chain name',
            description='Chain Name:',
            disabled=False
        )
        self.create_chain_spring_text = widgets.BoundedFloatText(
            min=0,
            max=1000000000,
            step=0.01,
            description='Spring Markup:',
            disabled=False
        )
        self.create_chain_summer_text = widgets.BoundedFloatText(
            min=0,
            max=1000000000,
            step=0.01,
            description='Summer Markup:',
            disabled=False
        )
        self.create_chain_fall_text = widgets.BoundedFloatText(
            min=0,
            max=1000000000,
            step=0.01,
            description='Fall Markup:',
            disabled=False
        )
        self.create_chain_winter_text = widgets.BoundedFloatText(
            min=0,
            max=1000000000,
            step=0.01,
            description='Winter Markup:',
            disabled=False
        )

        self.submit_create_chain = widgets.Button(
            description="Create Chain",
            disabled=False,
            button_style='success',
            tooltip="Create Chain",
            icon='check'
        )

        self.submit_delete_chain = widgets.Button(
            description="Delete Chain",
            disabled=False,
            button_style='danger',
            tooltip="Delete Chain",
            icon='trash'
        )

        self.submit_update_chain = widgets.Button(
            description="Update Chain",
            disabled=False,
            button_style='info',
            tooltip="Update Chain",
            icon='pencil'
        )

        self.status_label = widgets.Label()

        self.create_chain_container = widgets.VBox([
            self.create_chain_name_text,
            self.create_chain_spring_text,
            self.create_chain_summer_text,
            self.create_chain_fall_text,
            self.create_chain_winter_text,
            self.submit_create_chain,
            self.status_label
        ])

        self.delete_chain_container = widgets.VBox([
            widgets.Dropdown(
                options=chains_options,
                description='Select Chain to Delete:',
                disabled=False
            ),
            self.submit_delete_chain,
            self.status_label
        ])

        self.update_chain_container = widgets.VBox([
            widgets.Dropdown(
                options=chains_options,
                description='Select Chain to Update:',
                disabled=False
            ),
            self.create_chain_name_text,
            self.create_chain_spring_text,
            self.create_chain_summer_text,
            self.create_chain_fall_text,
            self.create_chain_winter_text,
            self.submit_update_chain,
            self.status_label
        ])

        self.submit_create_chain.on_click(self.on_submit_create_chain)
        self.submit_delete_chain.on_click(self.on_submit_delete_chain)
        self.submit_update_chain.on_click(self.on_submit_update_chain)

    def on_submit_create_chain(self, sender):
        name = self.create_chain_name_text.value
        spring = self.create_chain_spring_text.value
        summer = self.create_chain_summer_text.value
        fall = self.create_chain_fall_text.value
        winter = self.create_chain_winter_text.value
        data = {
            "cname": name,
            "springmkup": spring,
            "summermkup": summer,
            "fallmkup": fall,
            "wintermkup": winter
        }
        response = requests.post(f"{self.host}/deceptecons/chains", json=data)
        if response.status_code == 200:
            self.status_label.value = "Chain created successfully."
        else:
            self.status_label.value = "Failed to create chain."

    def on_submit_update_chain(self, sender):
        chain_id = self.update_chain_container.children[0].value.split(":")[0]
        name = self.create_chain_name_text.value
        spring = self.create_chain_spring_text.value
        summer = self.create_chain_summer_text.value
        fall = self.create_chain_fall_text.value
        winter = self.create_chain_winter_text.value
        data = {
            "cname": name,
            "springmkup": spring,
            "summermkup": summer,
            "fallmkup": fall,
            "wintermkup": winter
        }
        response = requests.put(f"{self.host}/deceptecons/chains/{chain_id}", json=data)
        if response.status_code == 200:
            self.status_label.value = "Chain updated successfully."
        else:
            self.status_label.value = "Failed to update chain."

    def on_submit_delete_chain(self, sender):
        chain_id = self.delete_chain_container.children[0].value.split(":")[0]
        response = requests.delete(f"{self.host}/deceptecons/chains/{chain_id}")
        if response.status_code == 200:
            self.status_label.value = "Chain deleted successfully."
        else:
            self.status_label.value = "Failed to delete chain."

def run(user, host, output, purpose):
    x = Chains(user, host)
    with output:
        if purpose == "Create":
            display(x.create_chain_container)
        elif purpose == "Delete":
            display(x.delete_chain_container)
        elif purpose == "Update":
            display(x.update_chain_container)


Chains
